In [22]:
from tqdm import tqdm
import jieba
# 调用 jieba分词module后，添加单词本（人名等）:
jieba.load_userdict("data/names_separate.txt")
import sys

# I. 数据处理和准备阶段

### 1） 输入原始文档

我的实验里面读入了《冰与火之歌》中文版的前5本，版权原因，我们不提供txt文件（各位可以自己搜索txt文件，逃。。。)


In [24]:
filename = 'data/ice_and_fire_utf8.txt'
text_lines = []

with open(filename, 'r') as f:
    for line in tqdm(f):
        text_lines.append(line)
print('总共读入%d行文字'% (len(text_lines)))

88765it [00:00, 1206561.24it/s]

总共读入88765行文字


### 2）调用jieba分词工具将句子切分成词语的序列

In [3]:
data_words, data_lines = [], []
# data_words: 训练我们的cbow和skip-gram模型
# data_lines: 调用gensim.word2vec训练word2vec模型

## 分词:
for line in tqdm(text_lines):
    one_line = [' '.join(jieba.cut(line, cut_all=False))][0].split(' ')
    data_words.extend(one_line)
    data_lines.append(one_line)

100%|██████████| 88765/88765 [00:14<00:00, 6199.62it/s]


### 3） 去掉标点和数字

In [5]:
import re
# 标点符号 (punctuation)
punct = set(u''':!),.:;?]}¢'"、。〉》」』】〕〗〞︰︱︳﹐､﹒﹔﹕﹖﹗﹚﹜﹞！），．＊：；Ｏ？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻︽︿﹁﹃﹙﹛﹝（｛“‘-—_…０１２３４５６７８９''')
isNumber = re.compile(r'\d+.*')

filter_words = [w for w in data_words if (w not in punct)
                and (not isNumber.search(w.lower()))]

### 4）建立词典

In [27]:
from collections import Counter

vocabulary_size = 30000
def build_vocab(words):
    """对文字数据中最常见的单词建立词典
    
    Arguments:
        words: 一个list的单词,来自分词处理过的文本数据库.
    
    Returns:
        data: 输入words数字编码后的版本
        count: dict, 单词 --> words中出现的次数
        dictionary: dict, 单词 --> 数字ID的字典
        reverse_dictionary: dict, 数字ID-->单词的字典
    """
    # 1. 统计每个单词的出现次数
    words_counter = Counter(filter_words)
    # 2. 选取常用词
    count = [['UNK', -1]]
    count.extend(words_counter.most_common(vocabulary_size - 1))
    # 3. 词语编号
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    # 4. 引入特殊词语UNK
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count = unk_count + 1
        data.append(index)
    
    print(unk_count)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reverse_dictionary


# 生成词典
data, count, dictionary, reverse_dictionary = build_vocab(filter_words)

41973


### 5） 打印一些信息，简单地人工检查有没有明显错误

In [15]:
print('共有 %d 万个词语' % (len(filter_words)//10000))
print('前十个单词是 %s' % ' '.join(filter_words[:10]))

共有 167 万个词语
前十个单词是 冰 与 火之歌 全集 卷 实体书 精校 版 
 作者


In [20]:
# 根据出现频率排列单词
from collections import Counter
words_counter = Counter(filter_words)
word_sorted = []
for (k,v) in words_counter.most_common(len(words_counter)):
    word_sorted.append((k,v))

# 看有哪些词，词频多少
# 是否常见词的词频高，生僻词的词频低
print('最常见的5个词，包括换行符:')
for i in range(5):
    print('%d: %s %d' % (i, word_sorted[i][0], word_sorted[i][1]))

print('\n')
print('出现频率居中的一些词语:')
for i in range(1111, 2000, 100):
    print('%d: %s %d' % (i, word_sorted[i][0], word_sorted[i][1]))

print('\n')
print('出现频率万名开外的一些词语')
for i in range(10000, len(word_sorted), 10000):
    print('%d: %s %d' % (i, word_sorted[i][0], word_sorted[i][1]))

最常见的5个词，包括换行符:
0: 的 91563
1: 
 88764
2: 他 39071
3: 我 28094
4: 了 24603


出现频率居中的一些词语:
1111: 才能 181
1211: 金 166
1311: 有时 154
1411: 记住 145
1511: 向前 135
1611: 那边 127
1711: 贝沃斯 120
1811: 发 113
1911: 工作 107


出现频率万名开外的一些词语
10000: 自相残杀 15
20000: 系到 5
30000: 下望 2
40000: 石头堆 1
50000: 森和卡 1
60000: 中见 1


In [25]:
# 验证输入jieba的一些name entity被正确地切分出来

demo_names = ['史塔克',
              '兰尼斯特',
              '龙之母']
for name in demo_names:
    print('%s 出现 %d 次'%(name, words_counter[name]))
    

史塔克 出现 1548 次
兰尼斯特 出现 1439 次
龙之母 出现 58 次


In [28]:
# 验证单词到数字的编码是正确的

demo_num = data[1000:1100]
print(demo_num)

demo_str = ''
for i in range(1000, 1100):
    demo_str = demo_str+(reverse_dictionary[data[i]])+' '
print(demo_str)

[2691, 195, 655, 778, 11, 1, 715, 6580, 19, 88, 11, 65, 3925, 3318, 510, 51, 137, 1, 3270, 232, 0, 847, 2691, 5, 533, 7008, 1019, 385, 51, 1, 2301, 7, 7240, 1, 449, 51, 86, 32, 11374, 11375, 362, 5857, 1, 2926, 2822, 711, 859, 11, 1457, 10252, 573, 106, 622, 1653, 5, 1617, 1, 2118, 13644, 2118, 1523, 778, 11, 119, 9374, 2118, 4724, 706, 1, 4224, 8976, 11376, 1160, 1160, 131, 5858, 517, 1, 15870, 7, 1554, 1, 29358, 51, 573, 106, 14671, 5, 231, 2220, 11, 860, 1, 3050, 232, 7, 711, 2691, 113, 1]
征服 之 路 尽管 他们 的 军队 数量 不 多 他们 却 有着 西方 世界 中 最后 的 三条 龙 UNK 此 征服 了 整个 大陆 七大 王国 中 的 六个 在 最初 的 战争 中 便 被 降服 唯独 多恩 激烈 的 反抗 以至于 伊耿 同意 他们 保持 独立 坦格利安 家族 同样 放弃 了 原来 的 信仰 改为 信仰 七神 尽管 他们 还是 违背 信仰 按照 瓦雷利亚 的 传统 兄妹 通婚   并 遵守 维斯特洛 的 风俗 在 接下来 的 数十年 中 坦格利安 家族 扑灭 了 所有 反对 他们 统治 的 叛乱 龙 在 伊耿 征服 后 的 


# II. 使用word2vec训练模型

In [29]:
import gensim
model = gensim.models.Word2Vec(iter=1)

In [30]:
model.build_vocab(data_lines)

In [31]:
model.train(data_lines, total_examples = len(data_lines), epochs = 10)

14361387

In [33]:
test_words = ['史塔克', '提利昂', '琼恩', '长城', '衣物', '力量', '没关系']
neighbors = []
for test_word in test_words:
    neighbors.append(model.most_similar(test_word))

In [34]:
for i in range(len(neighbors)):
    str = ' '.join([x[0] for x in neighbors[i]])
    print('%s:' % test_words[i])
    print('\t%s\n' % (str))

史塔克:
	徒利 波顿 艾林 葛雷乔伊 提利尔 兰尼斯特 卢斯 拉萨 戴林恩 佛雷

提利昂:
	詹姆 瑟曦 布蕾妮 侏儒 戴佛斯 瓦里斯 小指头 对方 珊莎 波隆

琼恩:
	山姆 席恩 耶哥蕊特 熊老 布兰 伊蒙学士 奈德 阿莎 艾莉亚 白灵

长城:
	巡逻 城墙 野人 守夜人 先民 拳峰 森林 远远 弟兄们 城堡

衣物:
	柴火 长裤 随身携带 钱币 雪橇 煮沸 泥巴 擦洗 带子 全盔

力量:
	生命 勇气 死亡 智慧 能力 代价 荣耀 胜利 信仰 人民

没关系:
	受不了 一辈子 不行 没用 无所谓 没差 不成问题 不怕 死活 用不着



# III. 使用我们的代码训练模型

In [35]:
import numpy as np
import tensorflow as tf
import collections

## III-1. skip-gram模型

In [36]:
data_index = 0
np.random.seed(0)

def generate_batch_sg(data, batch_size, num_skips, slide_window):
    
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * slide_window
    
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    
    # 滑动窗：[ slide_window target slide_window ]，宽度为 span
    span = 2 * slide_window + 1
    buffer = collections.deque(maxlen=span)
    
    # 扫过文本，将一个长度为 2*slide_window+1 的滑动窗内的词语放入buffer
    # buffer里面，居中的是target，“当前单词”
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    
    # # 下面的 for 循环：
    # 在产生一个有batch_size个样本的minibatch的时候，
    # 我们选择 batch_size//num_skips 个 “当前单词” （或者叫“目标单词”）
    # 并且从“当前单词”左右的2*slide_window 个词语组成的context里面选择
    # num_skips个单词
    # “当前单词”（我们叫做x）和num_skips个单词中的每一个（我们叫做y_i）
    # 组成一个监督学习样本：
    # 给定单词x, 在它的context里面应该大概率地出现单词y_i
    for i in range(batch_size // num_skips):
        # 在每个长度为2*slide_window + 1 的滑动窗里面，
        # 我们选择num_skips个（“当前单词”，“语境单词”）的组合
        # 为了凑齐batch_size个组合，我们需要batch_size//num_skips个滑动窗
        rand_x = np.random.permutation(span)
        j, k = 0, 0
        for j in range(num_skips):
            while rand_x[k]==slide_window:
                k += 1
            batch[i * num_skips + j] = buffer[slide_window]
            labels[i * num_skips + j, 0] = buffer[rand_x[k]]
            k += 1
        
        # 将滑动窗向右滑动随机步
        rand_step = np.random.randint(1,5)
        for _ in range(rand_step):
            buffer.append(data[data_index])
            data_index = (data_index + 1) % len(data)
        
    return batch, labels


In [42]:
# 测试代码：
batch_size = 8
for num_skips, slide_window in [(2, 1), (4,2)]:
    batch, labels = generate_batch_sg(data = data,
                                      batch_size=batch_size,
                                      num_skips=num_skips,
                                      slide_window=slide_window)
    print('\nwith num_skips = %d and slide_window = %d:' % (num_skips, slide_window))
    for i in range(batch_size):
        print('%s --> %s' % (reverse_dictionary[batch[i]],
                             reverse_dictionary[labels[i][0]]))


with num_skips = 2 and slide_window = 1:
UNK --> 成就
UNK --> 精心
的 --> 成就
的 --> 当今世界
上 --> 当今世界
上 --> 堪称
巅峰 --> 的
巅峰 --> 奇幻

with num_skips = 4 and slide_window = 2:
眼中 --> 冰
眼中 --> UNK
眼中 --> 们
眼中 --> 与
是 --> UNK
是 --> 与
是 --> 美国
是 --> 火之歌


In [44]:
import random
import math

batch_size = 128
num_sampled = 64 # 近似计算cross entropy loss的时候的negative examples参数
embedding_size = 128 # Dimension of the embedding vector

# skip-gram的两个重要的hyperparameters:
# 1. 语境范围：考虑和周围多少个词语的共存关系
slide_window = 1
# 2.　”使用频率“：（当前单词，临近单词）的组合使用多少次
num_skips = 1 # How many times to reuse an input to generate a label

# 产生测试数据
valid_size = 8
valid_examples = list(np.random.permutation(1000)[:valid_size])
names = ['史塔克', '提利昂', '琼恩', '长城', '南方', '死亡', '家族', '支持', '愤怒']
for name in names:
    valid_examples.append(dictionary[name])
    valid_size += 1

In [45]:
graph_sg = tf.Graph()

with graph_sg.as_default():#, tf.device('/cpu:0'):

    # Input data.
    train_dataset = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Variables.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    softmax_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))

    # Model.
    # Look up embeddings for inputs.
    embed = tf.nn.embedding_lookup(embeddings, train_dataset)
    # Compute the softmax loss, using a sample of the negative labels each time.
    loss = tf.reduce_mean(
        tf.nn.sampled_softmax_loss(weights=softmax_weights, biases=softmax_biases, inputs=embed,
                                   labels=train_labels, num_sampled=num_sampled, num_classes=vocabulary_size))

    # Optimizer.
    # Note: The optimizer will optimize the softmax_weights AND the embeddings.
    # This is because the embeddings are defined as a variable quantity and the
    # optimizer's `minimize` method will by default modify all variable quantities 
    # that contribute to the tensor it is passed.
    # See docs on `tf.train.Optimizer.minimize()` for more details.
    #optimizer = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(loss)
    optimizer = tf.train.AdagradOptimizer(learning_rate = 3.0).minimize(loss)

    # Compute the similarity between minibatch examples and all embeddings.
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(
        normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))

In [46]:
num_steps = 100001

with tf.Session(graph=graph_sg) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    average_loss = 0
    for step in range(num_steps):
        batch_data, batch_labels = generate_batch_sg(
            data, batch_size, num_skips, slide_window)
        feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
        _, l = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += l
        if step % 2000 == 0:
            if step > 0:
                average_loss = average_loss / 2000
            print('Average loss at step %d: %f' % (step, average_loss))
            average_loss = 0
        
        # note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in range(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8 # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k+1]
                log = 'Nearest to %s:' % valid_word
                for k in range(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log = '%s %s,' % (log, close_word)
                print(log)
    final_embeddings_sg = normalized_embeddings.eval()

Initialized
Average loss at step 0: 5.834519
Nearest to 不管: 不乏, 偏偏, 唯, 苦心经营, 联想, 一箭, 渴, 甚感,
Nearest to 戴佛斯: 洛克, 不看, 铸造, 容颜, 莱蒙, 龙穴, 设在, 能派,
Nearest to 亲自: 懒人, 让开, 奉承, 不, 追溯, 蓝宝石, 热, 荆棘女王,
Nearest to 看来: 不知疲倦, 眼眶里, 以示, 衰弱, 帷幕, 不佳, 木塞, 月门堡,
Nearest to 学士: 听人, 沿海, 甚感, 庞然, 仍然, 首度, 屁, 全完,
Nearest to 等: 他常, 招揽, 黑洞, 冰霜, 岸, 平安地, 不知, 背,
Nearest to 铁卫: ⑹, 起不来, 往外, 爱上你, 次女, 有知, 角, 两次,
Nearest to 维斯特洛: 贝壳, 火花, 银线, 乌云, 女人家, 份, 凿, 布偶,
Nearest to 史塔克: 喔, 屁滚尿流, 开溜, 闪现, 落坡, 退出, 偷来, 纹章,
Nearest to 提利昂: 好多年, 开朗, 铸造, 薄云, 渔民, 技士, 灰藓, 之际,
Nearest to 琼恩: 完好, 注意力, 蜜酒, 小麦, 吧, 罗纳德, 当学徒, 取笑,
Nearest to 长城: 额间, 同名, 啤酒, 冷风, 入河, 方面, 群星, 铁民是,
Nearest to 南方: 探视, 美丽动人, 逃兵, 并肩而行, 荒山, 小床, 踹, 果肉,
Nearest to 死亡: 人喊, 靠岸, 拐棍, 白玫瑰, 芭蕾, 出彩, 暴雪, 污染,
Nearest to 家族: 温家, 打趣, 房产, 千万别, 四座, 走投无路, 水边, 悔不当初,
Nearest to 支持: 哈格, 自始至终, 目瞪口呆, 无语, 水手, 绰号, 有过之而无不及, 纵声,
Nearest to 愤怒: 女泉, 属, 趁此机会, 怒不可遏, 以图, 大忙, 关上门, 真不错,
Average loss at step 2000: 4.464754
Average loss at step 4000: 4.118656
Average loss at step 6000: 3.976826
Average loss a

KeyboardInterrupt: 

### III-2. cbow 模型

In [87]:
data_index = 0

def generate_batch_cbow(data, batch_size, slide_window, num_skips):
    """
    Arguments:
        data: list_of_integer格式的文本
        batch_size: 一个minibatch的样本大小
        slide_window: 滑动窗口的大小，定义语境（context）的范围
        num_skips: 语境(context)包含的单词数量
        
    Returns:
        batch: 一个minibatch的语境
        labels: 一个minibatch的标记
    """
    
    global data_index
    
    # 和skip-gram不同，
    # cbow里面，输入的语境context 和 输出的label包含不同数量的单词
    # 因为，一个样本包含 (1) num_skips个context单词 和 (2) 一个label单词
    # 我们表示为 num_context 和 batch_size：
    num_context = num_skips * batch_size
    batch = np.ndarray(shape=(num_context), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    
    # # 使用一个buffer记录和更新滑动窗，[ slide_window target slide_window ]
    span = 2 * slide_window + 1 
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    
    # # 从每一个新的滑动窗提取一个训练样本:
    # （num_skips个单词组成的语境context, 一个目标单词作为标记）
    for i in range(batch_size):
        rand_x = np.random.permutation(span)
        if 2 * slide_window == num_skips:
            for j in range(slide_window):
                batch[i*num_skips + j] = buffer[j]
            for j in range(slide_window, 2 * slide_window):
                batch[i*num_skips + j] = buffer[j+1]
        else:
            j, k = 0, 0
            for j in range(num_skips):
                while rand_x[k]==slide_window:
                    k += 1
                batch[i * num_skips + j] = buffer[rand_x[k]]
                k += 1
        labels[i, 0] = buffer[slide_window]
        
        # 将滑动窗向右滑动随机步
        rand_step = np.random.randint(1,5)
        for _ in range(rand_step):
            buffer.append(data[data_index])
            data_index = (data_index + 1) % len(data)
    
    return batch, labels


In [98]:
# 测试：
slide_window = 1
num_skips = 2
batch_size = 8

batch, labels = generate_batch_cbow(data = data,
                                    batch_size = batch_size,
                                    slide_window = slide_window,
                                    num_skips = num_skips)

print('\nwith num_skips = %d and slide_window = %d:' % (num_skips, slide_window))

print('batch context size = %s' % repr(np.shape(batch)))
for x in range(0, batch_size * num_skips, num_skips):
    print(' '.join([reverse_dictionary[bi] for bi in batch[x:x+num_skips]])
          + ' ==> '
          + reverse_dictionary[labels[x//num_skips][0]])



with num_skips = 2 and slide_window = 1:
batch context size = (16,)
不像 魔幻 ==> UNK
魔幻 一样 ==> 小说
小说 用 ==> 一样
的 或是 ==> 魔法
物种 吸引 ==> 来
人们 眼球 ==> 的
而是 跌宕起伏 ==> 讲究
跌宕起伏 情节 ==> 的


In [67]:
batch_size = 128
embedding_size = 128 # 词向量维度
num_sampled = 64   # Number of negative examples to sample.

## cbow的hyperparameters：
# 语境范围，考虑和周围多少个词语的共存关系
slide_window = 1
num_skips = 2

# 产生测试数据
valid_size = 8
valid_examples = list(np.random.permutation(1000)[:valid_size])
names = ['史塔克', '提利昂', '琼恩', '长城', '南方', '死亡', '家族', '支持', '愤怒']
for name in names:
    valid_examples.append(dictionary[name])
    valid_size += 1


In [99]:

graph_cbow = tf.Graph()
with graph_cbow.as_default():
    
    ## 第一层：数据
    # 训练数据分配内存空间
    train_dataset = tf.placeholder(tf.int32, shape=[num_skips * batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    #　固定的测试数据，以tf.constant这种variable形式
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    ## 第一层与第二层之间：词向量矩阵
    # 一个tf.Variable对象，将整数ID映射为词向量特征
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    
    ## 第二层：通过查询操作将context（整数ID的list）转化为词向量
    # 通过对context里面的所有词的词向量们简单加和得到context的词向量表示
    # shape = [ (batch_size * num_skip) x embedding_size ]
    embed = tf.nn.embedding_lookup(embeddings, train_dataset)
    # shape = [ batch_size x num_skip x embedding_size ]
    embed = tf.reshape(embed, [batch_size, num_skips, -1])
    # shape = [ batch_size x embedding_size]
    final_embed = tf.reduce_sum(input_tensor = embed,
                          axis = 1,
                          keep_dims = False)
    
    ## 第三层：预测
    # 给定context的词向量表示，预测target，和真实的target比较，计算loss
    softmax_weights = tf.Variable(tf.truncated_normal(
        [vocabulary_size, embedding_size],
        stddev=1.0 / math.sqrt(embedding_size)))
    softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))
    
    loss = tf.reduce_mean(
        tf.nn.sampled_softmax_loss(weights = softmax_weights,
                                   biases = softmax_biases,
                                   inputs = final_embed,
                                   labels = train_labels,
                                   num_sampled = num_sampled,
                                   num_classes = vocabulary_size))
    
    # 优化参数
    #optimizer = tf.train.AdamOptimizer(learning_rate = 0.01).minimize(loss)
    optimizer = tf.train.AdagradOptimizer(learning_rate = 1.0).minimize(loss)
    
    ## 测试模型：
    #　
    # Compute the similarity between minibatch validation examples and
    #   all embeddings, using the cosine distance:
    # 
    # 第一步： 对每个单词的词向量归一化
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    # 小心： must use argument 'keep_dims=True' to normalize correctly
    normalized_embeddings = embeddings // norm
    # 第二步： 对 valid_dataset 里面的每一个单词查询归一化的词向量
    valid_embeddings = tf.nn.embedding_lookup(
        normalized_embeddings, valid_dataset)
    # 第三步： 通过内积计算 valid_dataset 里面的每一个单词和所有单词的cosine similarity
    similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))

In [101]:
num_steps = 100001

with tf.Session(graph=graph_cbow) as session:
    session.run(tf.global_variables_initializer())
    print("Initialized")
    average_loss = 0
    for step in range(num_steps):
        # generate minibatch of samples, feed into feed_dict
        batch_data, batch_labels = generate_batch_cbow(data = data,
                                    batch_size = batch_size,
                                    slide_window = slide_window,
                                    num_skips = num_skips)
        
        feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
        
        # run session to calculate loss and optimize parameters
        _, l = session.run([optimizer, loss], feed_dict=feed_dict)
        
        # 1. simple evaluation:
        #   calculate and display average loss every 2000 minibatch
        average_loss += l
        if step % 2000 == 0:
            if step > 0:
                average_loss = average_loss / 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print("Average loss at step %d is %s" % (step,average_loss))
            average_loss = 0
        
        # 2. expensive evaluation:
        #   calculate cosine similarities every 10,000 minibatch
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in range(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8 # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k+1]
                log = "Nearest to %s:" % valid_word
                for k in range(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log = "%s %s," % (log, close_word)
                print(log)
    final_embeddings_cbow = normalized_embeddings.eval()

Initialized
Average loss at step 0 is 8.7382516861
Nearest to 所: 矛兵, 绷起, 摇身一变, 身材矮小, 搁下, 呵欠, 越趋, 宠幸,
Nearest to 头: 粗大, 高墙, 一眼, 之牙上, 不见, 先要, 身材矮小, 波纹,
Nearest to 认为: 红铜, 箭矢, 好时机, 偷笑, 箭靶, 北风, 矮马, 撮,
Nearest to 转向: 拍打着, 八九天, 毫不犹豫, 我命, 故而, 睡不醒, 征税, 蹄铁,
Nearest to 却: 亲随, 大营, 宝宝, 主子, 交给, 之神, 欣慰, 山头,
Nearest to 战争: 症状, 睡着, 夭亡, 矛兵, 矮种, 糟糕透顶, 不见, 菌类,
Nearest to 一直: 妓魇安柏, 波动, 磨得, 林子, 摇醒, 生涯, 费力, 用龙,
Nearest to 一道: 银舌西蒙, 宝宝, 不生, 杀出, 办法, 八九天, 大营, 往下走,
Nearest to 史塔克: 干酪, 往下走, 睡不醒, 止血, 摇醒, 辨出, 骇人, 恰当,
Nearest to 提利昂: 许诺, 海龟, 心中, 摇醒, 几句话, 厚度, 止血, 饕餮,
Nearest to 琼恩: 禽兽, 干酪, 症状, 罪孽, 糟糕透顶, 喝令, 宝宝, 飞离,
Nearest to 长城: 摇醒, 拆除, 硬拖, 老得, 布鲁斯, 好容易, 木盾, 怒道,
Nearest to 南方: 湖中, 胆大包天, 老得, 我哥, 龙太子, 河间, 伤腿, 木柄,
Nearest to 死亡: 面露, 往下走, 不会错, 只算, 石头路, 富丽堂皇, 由, 洗涤,
Nearest to 家族: 一簇, 帮帮, 发落, 飘落, 发着, 疹子, 倦怠, 矛兵,
Nearest to 支持: 防御工事, 莫名其妙, 别管, 老外, 扳机, 黑焰, 摇醒, 放鹰,
Nearest to 愤怒: 轻侮, 摇醒, 真多, 衣领, 地探, 乱打, 派娅妮, 深灰色,
